In [1]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [7]:
HV_ccg = []
with open('./path/to/HighlyVariableCellCycleGeneslist.txt',mode="r") as cycle:
    for i in cycle:
        HV_ccg.append(i[:-1])
Regulators = ['MYC', 'SMAD2', 'TFDP1', 'TFDP2','EP300','CCNA2','CCNH','CDK1','HDAC1','HDAC2','MAD2L2','MDM2','PLK1','PTTG1','RAD21','RBX1','YWHAB']

In [10]:
GRNs = {}
for c in range(1,16):
    ckey = "Cell"+str(c)
    GRN_df = pd.read_csv("./path/to/GRN_{}".format(ckey),index_col=0)
    for g in HV_ccg:
        if g not in GRN_df.index:
            GRN_df.loc[g] = np.zeros(len(GRN_df.columns))
    GRN_df = GRN_df.loc[Regulators]
    for g in HV_ccg:
        if g not in GRN_df.columns:
            GRN_df[g] = np.zeros(len(GRN_df.index)) 
    GRNs[ckey] = GRN_df/GRN_df.max().max()

In [ ]:
fluc = {}
for reg in Regulators:
    for resp in HV_ccg:
        if reg == resp:
            continue
        fluc[reg+">"+resp] = []
        for cell_key in range(1,16):
            ckey = "Cell"+str(cell_key)
            if reg in GRNs["Cell{}".format(cell_key)].index and resp in GRNs["Cell{}".format(cell_key)].columns:
                fluc[reg+">"+resp].append(GRNs["Cell{}".format(cell_key)].loc[reg,resp])
            else:
                fluc[reg+">"+resp].append(0)
fluc_cv = {}
for combi,w in fluc.items():
    fluc_cv[combi] = np.var(w)/np.mean(w)
    #fluc_var[combi] = np.var(w)
fluc_cv_df_raw = pd.DataFrame(fluc_cv,index=["cv"]).T.sort_values("cv",ascending=False)
fluc_cv_df = pd.DataFrame(stats.zscore(fluc_cv_df_raw),index=fluc_cv_df_raw.index,columns=fluc_cv_df_raw.columns).sort_values("cv",ascending=False)

fluc_var = {}
for combi,w in fluc.items():
    fluc_var[combi] = np.var(w)
fluc_var_df_raw = pd.DataFrame(fluc_var,index=["var"]).T.sort_values("var",ascending=False)
fluc_var_df = pd.DataFrame(stats.zscore(fluc_var_df_raw),index=fluc_var_df_raw.index,columns=fluc_var_df_raw.columns).sort_values("var",ascending=False)

sums = {}
for combi,w in fluc.items():
    sums[combi] = np.mean(w)
sum_df_raw = pd.DataFrame(sums,index=["sum"]).T.sort_values("sum",ascending=False)
sum_df = pd.DataFrame(stats.zscore(sum_df_raw),index=sum_df_raw.index,columns=sum_df_raw.columns).sort_values("sum",ascending=False)

In [ ]:
unstacked = np.array([])
for c in range(1,16):
    ckey = "Cell"+str(c)
    if c == 1:
        unstacked = GRNs[ckey].unstack().values
    else:
        unstacked = np.vstack([unstacked,GRNs[ckey].unstack().values])
grn_corr = pd.DataFrame(unstacked).T.corr(method="spearman")
grn_corr.index = ["Cell"+str(i) for i in range(1,16)]
grn_corr.columns = ["Cell"+str(i) for i in range(1,16)]

In [ ]:
fucci = pd.read_csv("./path/to/FucciInt_LiveCell.csv",index_col=0)
fucci.columns=["Green","Red","Time","Cell"]
dist_dic_alt = {}
for c1 in range(1,16):
    ckey1 = "Cell"+str(c1)
    dist_dic_alt[ckey1] = dict()
    for c2 in range(1,16):
        ckey2 = "Cell"+str(c2)
        dist = np.sum(np.sqrt((fucci[fucci["Cell"]==ckey1]["Green"].values - fucci[fucci["Cell"]==ckey2]["Green"].values)**2
                              +(fucci[fucci["Cell"]==ckey1]["Red"].values - fucci[fucci["Cell"]==ckey2]["Red"].values)**2))
        dist_dic_alt[ckey1][ckey2] = dist
dist_df_o = pd.DataFrame(dist_dic_alt)
dist_df_o = dist_df_o/dist_df_o.max().max()


In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram
result1 = linkage(grn_corr, metric = 'euclidean',method = 'ward')
den = dendrogram(result1)
plt.title("Dedrogram")
plt.ylabel("Threshold")
plt.show()
from sklearn.cluster import AgglomerativeClustering
hir_clus   = AgglomerativeClustering(n_clusters = 2, affinity = 'euclidean', linkage = 'ward')
hist_order = [grn_corr.columns[int(i)] for i in den["ivl"]]

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
grad = ["#294C60","#437C9D","#7899D1","#DFDFDF","#BB9493","#AC6A69","#9D4343","#7F3737"]
cmap_rb = LinearSegmentedColormap.from_list('custom',grad)
cmap_rb_r = LinearSegmentedColormap.from_list('custom',grad[::-1])

In [ ]:
dist_df = dist_df_o.loc[hist_order,hist_order]
mask = np.triu(np.ones_like(dist_df.values, dtype=np.bool))
fig = plt.figure(figsize=(1.5,1.5))
ax1 = fig.add_subplot(111)
sns.heatmap(dist_df, mask=mask, annot=False, fmt=".2f", cmap=cmap_rb_r,#cmap_new_r,
            cbar=False,
           cbar_kws={"shrink": .8},square=True,ax=ax1,linewidth=0.5)
ax1.axes.xaxis.set_ticks([])
ax1.axes.yaxis.set_ticks([])

In [ ]:
grn_corr = grn_corr.loc[hist_order,hist_order]
fig = plt.figure(figsize=(1.5,1.5))
#plt.ylim(0, scd_ma.max())
ax1 = fig.add_subplot(111)
mask = np.triu(np.ones_like(grn_corr.values, dtype=np.bool))
sns.heatmap(np.array(grn_corr), mask=mask, annot=False, fmt=".2f", cmap=cmap_rb,
           cbar_kws={"shrink": .8},square=True,ax=ax1,linewidth=0.5, cbar=False)
ax1.axes.xaxis.set_ticks([])
ax1.axes.yaxis.set_ticks([])
plt.show()

In [ ]:
dist_fucci = []
corrs = []
for i in range(1,16):
    for j in range(i+1,16):
        key1 = "Cell"+str(i)
        key2 = "Cell"+str(j)
        corrs.append(grn_corr.loc[key1,key2])
        dist_fucci.append(dist_df_o.loc[key1,key2])
CR = np.corrcoef(corrs,dist_fucci)[0][1]
import matplotlib as mpl
font = {'family' : 'Arial','size': 7}
mpl.rc('font', **font)
fig = plt.figure(figsize=(1.2,1.2))
ax1 = fig.add_subplot(111)
ax1.xaxis.grid(True, which='major', linestyle='--', color ="lightgray",linewidth=0.5)
ax1.yaxis.grid(True, which='major', linestyle='--', color ="lightgray",linewidth=0.5)
ax1.set_axisbelow(True)
ax1.tick_params(labelbottom="off",bottom="on", labelleft="on",left="on", length=2,width=1, color=black)
#ax2.tick_params(labelbottom="on",bottom="on", labelleft="on",left="on", length=1, color=black)
for j in ["left","bottom"]:
    ax1.spines[j].set_color(black)
for j in ["top","right"]:
    ax1.spines[j].set_color("w")
    #ax1.spines[j].set_width(1)
df = pd.DataFrame([(np.array(corrs)), dist_fucci], index=["G","F"]).T
#sns.regplot(data = df,x="G",y="F", scatter_kws={'s':5,"edgecolor":"w","linewidth":0,"alpha":1},
#            color="#5C5C5C", line_kws={"color": blue,"linewidth":1.5})#,ax=ax1)
df.plot.scatter(x="G",y="F",s=3,ax=ax1,color="dimgray")
#polr_gradient
plt.xlabel("")
plt.ylabel("")
#ax1.scatter(x=corrs,y=dist_fucci,s=12.5,color=colors_15[-3],edgecolor=black,linewidth=0.5)

ax1.xaxis.grid(True, which='major', linestyle='--', color ="lightgray",linewidth=0.5)
ax1.yaxis.grid(True, which='major', linestyle='--', color ="lightgray",linewidth=0.5)

plt.show()
print(CR)
print(stats.spearmanr(corrs,dist_fucci))